# Data Science - Python and Pandas


## Table of Content

1. [Introduction](#introduction)<br>
1.1. [Series and DataFrames](#series)<br>
1.2. [Indexing](#index)<br>
1.3. [Data Selection](#selection)<br>
2. [Transform data](#transform)<br>
2.1. [Adding and deleting columns](#columns)<br>
2.2. [Cleaning Data](#cleaning)<br>
2.3. [Merging Data](#merging)<br>
2.4. [Grouping Data](#grouping)<br>
3. [Visualise data](#visualise)<br>
4. [Optional Excercises](#extra)<br>



Let's start with loading the packages and a quick look at some data. Select the below cell by clicking on it, and then click on the `Run` button at the top of the notebook (or use `Shift+Enter`). This is how you can run all code cells in this notebook. The numbers in front of the cells tell you in which order you have run them, for instance `[1]`. When you see a `[*]` the cell is currently running and `[]` means you have not run the cell yet. 

In [ ]:
!pip install --upgrade seaborn

After running the first cell with code above, restart the notebook by clicking on the `Kernel` tab at the top of the notebook, and then `Restart`. You do not have to run the above cell again after the restart as updating the seaborn package only has to be done once. Then run the next cell that will import two other packages:

In [ ]:
import numpy as np
import pandas as pd

Loading data from Cloud Object Store (COS) is done by adding the `measurements.csv` file in the menu on the right of the notebook (if you see no menu, click the `1010` button at the top first). 

- Activate the below cell, move the cursor to the empty line under `# add data`
- Click on `Insert to code` under the file from the right menu
- Select `Insert pandas DataFrame`
- Code to load the file will be inserted
- Change the default name of the data from `df_data_1` to `jeans` at the bottom two rows of the inserted code

In [ ]:
# add data
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.
client_0ca3522ddd5d4c97a16d00548d7b6f62 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='X-cc9g-AMbOrbySPm9RorHR0SsQtfXJ2_laj3if2jaLi',
    ibm_auth_endpoint="https://iam.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_0ca3522ddd5d4c97a16d00548d7b6f62.get_object(Bucket='geopandasworkshop-donotdelete-pr-m4tagpbneu4txf',Key='jeans_measurements.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

jeans = pd.read_csv(body)
jeans.head()



In [ ]:
# If you want to  run this notebook locally use:
#jeans = pd.read_csv('jeans_measurements.csv')

Now let's have a look at the data that has been loaded into the notebook. Use `jeans`, `jeans.head()` or `jeans.tail()` to see different parts of the table and `jeans.dtypes` to see which variables there are and what datatype they have. Add a number between the brackets `()` to specify how many lines you want to display.

> _Tip_: If you want to run these in separate cells, activate the below cell by clicking on it and then click on the `+` at the top of the notebook. This will add extra cells. Click on the upwards and downwards arrows to move the cells up and down to change their order. 

In [ ]:
jeans

<div class="alert alert-success">
 <b>EXERCISE</b> <br/> 
 Explore the following commands:
  <ul>
  <li><font face="Courier">jeans.columns</font></li>
  <li><font face="Courier">jeans.values</font></li>
  <li><font face="Courier">jeans.shape</font></li>
  <li><font face="Courier">len(jeans)</font></li>
  
 </ul> 
</div>  

In [ ]:
# try the commands here (add as many cells as you need):


<a id="introduction"></a>
## 1. Introduction

The Python package you used to read this file and look at some of it's properties is [Pandas](https://pandas.pydata.org/), which is an open source library with easy-to-use data structures and data analysis tools. 

<div class="alert alert-info" style="font-size:100%">
<b>Read this <a href="http://pandas.pydata.org/pandas-docs/stable/getting_started/10min.html">10 minute introduction</a> for a quick overview of Pandas.<br>
</div>

<a id="series"></a>
### 1.1 Series and DataFrames 

Let's go through some of the basics of Pandas before going back to the Jeans dataset. Pandas has two main data structures: `Series` and `DataFrames`. 

A `Series` is a list of values with an integer index. The first column is the index (the default starts at 0) and the second column the values.

In [ ]:
s = pd.Series([1, 3, 5, np.nan, 6, 8])
s

 A `DataFrame` is similar, but has multiple columns. You can create one in many ways, by loading a file or from for example a NumPy array and a date for the index. (We come back to the index and dates later) 


<div class="alert alert-info" style="font-size:100%">
<b>Read this <a href="https://docs.scipy.org/doc/numpy-1.15.0/user/quickstart.html"> tutorial</a> for an overview of NumPy.<br>
</div>

Two examples:

In [ ]:
dates = pd.date_range('20130101', periods=6)
dates

In [ ]:
numbers = np.random.randn(6, 4)
numbers

In [ ]:
df = pd.DataFrame(numbers, index=dates, columns=list('ABCD'))
df

In [ ]:
df2 = pd.DataFrame({'A': 1.,
                     'B': pd.Timestamp('20130102'),
                     'C': pd.Series(1, index=list(range(4)), dtype='float32'),
                     'D': np.array([3] * 4, dtype='int32'),
                     'E': pd.Categorical(["test", "train", "test", "train"]),
                     'F': 'foo'})

In [ ]:
df2.head()

To find out what the data type is of a variable use `type()`: 

In [ ]:
print('Data type of s is '+str(type(s)))
print('Data type of s is '+str(type(dates)))
print('Data type of s is '+str(type(numbers)))
print('Data type of df is '+str(type(df)))

In [ ]:
type(jeans)

<a id="index"></a>
### 1.2 Indexing 

It is important to understand the `index` to work with dataframes, so let's explore this a little further. 

For this we will create a new DataFrame with the population of the 5 largest cities in the UK ([source](https://en.wikipedia.org/wiki/List_of_urban_areas_in_the_United_Kingdom)). `data` is a [dictionary](https://realpython.com/python-dicts/).

In [ ]:
data = {'city':       ['London','Manchester','Birmingham','Leeds','Glasgow'],
        'population': [9787426,  2553379,     2440986,    1777934,1209143],
        'area':       [1737.9,   630.3,       598.9,      487.8,  368.5 ]}
cities = pd.DataFrame(data)
cities

In [ ]:
cities.columns

In [ ]:
cities.index

In [ ]:
cities = cities.set_index('city')
cities

In [ ]:
cities = cities.reset_index()
cities = cities.set_index(['city','population'])
cities

In [ ]:
cities = cities.reset_index()
cities = cities.set_index('city')
cities

<a id="selection"></a>
### 1.3 Data Selection

Select a single label or a range of labels with `.loc[]` (This only works for the column that was set to the index):

In [ ]:
cities.loc['London', 'area']

In [ ]:
cities.loc['Manchester':'Leeds', ['area', 'population']]

Or select by position with `.iloc[]`. You can select a single row, multiple rows (or columns) at particular positions in the index, it only takes integers:

In [ ]:
cities.iloc[0]

In [ ]:
cities.iloc[:,1]

In [ ]:
cities.iloc[:,0:2]

In [ ]:
cities.iloc[2:4,0:2]

You can also use one or more column names to create a new DataFrame.

In [ ]:
cities['area']

In [ ]:
cities2 = cities[['area','population']]
cities2

#### Filtering

Selecting rows based on a certain condition can be done with Boolean indexing:

In [ ]:
cities['area'] > 500

If you want to select the data add `cities[]` around the above:

In [ ]:
cities[cities['area'] > 500]

Combining different columns using `&`, `|` and `==` is also possible"

In [ ]:
cities[(cities['area'] > 500) & (cities['population'] > 2500000)]

In [ ]:
cities[(cities['area'] < 500) | (cities['population'] < 1000000)]

In [ ]:
cities[cities['area'] == 487.8] 

<div class="alert alert-success">
 <b>EXERCISE</b> <br/> 
 Using the jeans DataFrame:
  <ul>
  <li>Find the most expensive jeans</li>  
  <li>Find the cheapest jeans</li>
  <li>What is the difference in price between the cheapest and most expensive jeans?</li>    
 </ul>  
</div>  

> *Tips*: 
- Find the maximum of a row with for instance `jeans['price'].max()` 
- Extract the value from a cell in a DataFrame with `.value[]`
- To see the answer uncomment the line in the cells that contains `%load` (by deleting the `#`) and then run the cell, but try to find your own solution first in the cell above the solution!

In [ ]:
# most expensive jeans


In [ ]:
# %load https://raw.githubusercontent.com/IBMDeveloperUK/pandas-workshop/master/answers/answer1.py

In [ ]:
# cheapest jeans


In [ ]:
# %load https://raw.githubusercontent.com/IBMDeveloperUK/pandas-workshop/master/answers/answer2.py

In [ ]:
# difference in price


In [ ]:
# %load https://raw.githubusercontent.com/IBMDeveloperUK/pandas-workshop/master/answers/answer3.py

<a id="transform"></a>
## 2. Transform Data

When looking at data there are always transformations needed to get it in the format you need for your analysis, visualisations or models. 

These are a few examples of the endless possibilities. The best way to learn is to find a dataset and try to answer questions with the data. The pandas documentation is real good, and on StackOverflow there is almost always someone who asked the same question already. 

<a id="columns"></a>
### 2.1 Adding and deleting columns
Adding a column can be done by defining a new column, which can then be dropped with 'drop'. 

In [ ]:
jeans['new'] = 1
jeans = jeans.drop(columns='new')

In [ ]:
jeans['avgHeightFront'] = (jeans.maxHeightFront + jeans.minHeightFront) / 2

In [ ]:
jeans.head()

<a id="cleaning"></a>
### 2.2 Cleaning Data

Things to check:

- Is the data tidy: each variable forms a column, each observation forms a row and  each type of observational unit forms a table.
- Are all columns in the right data format?
- Are there missing values?
- Are there unrealistic outliers?

Get a quick overview of the numeric data with `.describe()`. If any of the numeric columns is missing this is a probably because of a wring data type. 


In [ ]:
jeans.describe()

It is not always ideal to have text in the table. Especially not if you want to create a model from the data. You could replace `style` into numbers, but is one style really twice as large as another. It is better to transform the data with `get.dummies()`. The below will add 4 new columns to the DataFrame:

In [ ]:
jeans.head()

In [ ]:
jeans2 = jeans.copy()
style = pd.get_dummies(jeans2['style'], drop_first=True)
jeans2 = jeans2.join(style)
jeans2.head(2)

Or do this all in one line of code:

In [ ]:
jeans = jeans.join(pd.get_dummies(jeans['style'], drop_first=True))
jeans.head(2)

<a id="merging"></a>
### 2.3 Merging Data

There are several ways to combine data. The [documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html) has lots of examples. You can combine data with `.append()` or `.concat()`:

In [ ]:
data = {'city':       ['London','Manchester','Birmingham','Leeds','Glasgow'],
        'population': [9787426,  2553379,     2440986,    1777934,1209143],
        'area':       [1737.9,   630.3,       598.9,      487.8,  368.5 ]}
cities = pd.DataFrame(data)

data2 = {'city':       ['Liverpool','Southampton'],
        'population': [864122,  855569],
        'area':       [199.6,   192.0]}
cities2 = pd.DataFrame(data2)

These new cities can be added with `append()`:

In [ ]:
cities = cities.append(cities2)
cities

In [ ]:
data = {'city': ['London','Manchester','Birmingham','Leeds','Glasgow'],
        'density': [5630,4051,4076,3645,3390]}
cities3 = pd.DataFrame(data)

In [ ]:
cities3

In [ ]:
cities = pd.merge(cities, cities3, how='outer', sort=True,on='city')
cities

Data that does not quite fit can be merged as well:

In [ ]:
data = {'city':       ['Newcastle','Nottingham'],
        'population': [774891,  729977],
        'area':       [180.5,   176.4]}

cities4 = pd.DataFrame(data)
cities4

In [ ]:
cities = cities.append(cities4)
cities

<a id="grouping"></a>
### 2.4 Grouping Data


In [ ]:
jeans.columns

In [ ]:
jeans.groupby(['cutout']).mean()

In [ ]:
jeans.groupby(['cutout','style']).max().head(10)

<div class="alert alert-success">
 <b>EXERCISES</b> <br/> 
 Using the jeans DataFrame:
  <ul>
  <li>Add a column `FrontArea` with the area of the front pocket (height X width) </li>        
  <li>Add a column `BackArea` with the area of the back pocket (height X width) </li>        
  <li>Add two columns `men` and `women` with `get_dummies()` and keep the original `menWomen`</li>        
  <li>Using `groupby().count()`: what is the number of mens and womens jeans measured?</li>        
  <li>What are the average front and back pocket sizes of mens and womens jeans?</li>   
 
 </ul>  
</div>  

> *Tips*: 
- To find out how many unique values there are in a column use `np.unique(df['a'])`
- You can use `mean()`, `max()`, `min()`, `count()` and more with `groupby()`

In [ ]:
# FrontArea


In [ ]:
# %load https://raw.githubusercontent.com/IBMDeveloperUK/pandas-workshop/master/answers/answer4.py

In [ ]:
# BackArea


In [ ]:
# %load https://raw.githubusercontent.com/IBMDeveloperUK/pandas-workshop/master/answers/answer5.py

In [ ]:
# men and women columns


In [ ]:
# %load https://raw.githubusercontent.com/IBMDeveloperUK/pandas-workshop/master/answers/answer6.py

In [ ]:
# number of mens and womens jeans 


In [ ]:
# %load https://raw.githubusercontent.com/IBMDeveloperUK/pandas-workshop/master/answers/answer7.py

In [ ]:
# average front and back pocket sizes for men and womens jeans


In [ ]:
# %load https://raw.githubusercontent.com/IBMDeveloperUK/pandas-workshop/master/answers/answer8.py

<a id="explore"></a>
## 3. Visualizing Data

In [ ]:
# without this the plots would be opened  in a new window (not browser)
# with this instruction plots will be included in the notebook
%matplotlib inline

import matplotlib.pyplot as plt

default
uses matplotlib

In [ ]:
jeans['price'].plot();

In [ ]:
jeans['price'].plot.hist(bins=3);

In [ ]:
jeans['price'].plot.hist(bins=15,figsize=(10,5));

In [ ]:
jeans['price'][jeans['menWomen']=='men'].plot.hist(bins=15,figsize=(10,5));

In [ ]:
jeans['price'][jeans['menWomen']=='men'].plot.hist(bins=15,figsize=(10,5));
jeans['price'][jeans['menWomen']=='women'].plot.hist(bins=15,figsize=(10,5));

In [ ]:
ax = jeans['price'][jeans['menWomen']=='men'].plot.hist(
    bins=15,figsize=(10,5),alpha=0.5,color='#1A4D3B');
ax = jeans['price'][jeans['menWomen']=='women'].plot.hist(
    bins=15,figsize=(10,5),alpha=0.5,color='#4D1A39');
ax.legend(['men','women']);

In [ ]:
jeans['price'].plot.hist(
    bins=15, 
    title="Jeans Price",
    legend=False,
    fontsize=14,
    grid=False,
    linestyle='--',
    edgecolor='black',
    color='darkred',
    linewidth=3);

In [ ]:
style = jeans['price'].groupby(jeans['style']).mean()
ax=style.plot.bar();
ax.set_ylabel('Jeans Price');

## Seaborn

Seaborn is an easy to use visualisation package that works well with Pandas DataFrames. Below are a few examples, but have a look at the [documentation](https://seaborn.pydata.org/index.html) as there are many more plots you could make. 

In [ ]:
import seaborn as sns

In [ ]:
sns.distplot(jeans['price']);

In [ ]:
sns.distplot(np.array(jeans['price']));

In [ ]:
sns.catplot(x='menWomen', y='price', data=jeans);

In [ ]:
sns.catplot(x='menWomen', y='price', hue='style', kind='swarm', data=jeans);

In [ ]:
sns.catplot(x="style", y="price", kind="box", data=jeans);

In [ ]:
sns.catplot(x="style", y="price", hue="menWomen", kind="box", data=jeans);

<div class="alert alert-success">
 <b>EXERCISE</b>
 <ul>
  <li>Create two histograms that compare the sizes of pockets between men and womens jeans with `.plot.hist()`</li>
  <li>Create a bar plot with the size of the front pockets for men and women with `.plot.bar()`</li>
  <li>Create a bar plot with the size of the front pockets for men and women with `seaborn`</li>
  <li>Customize the way the example or your plots look</li>
 </ul> 
</div>    

 
> Tip: to add two histograms to one plot you can repeat `.plot()` in the same cell 


In [ ]:
# histogram front pockets


In [ ]:
# %load https://raw.githubusercontent.com/IBMDeveloperUK/pandas-workshop/master/answers/answer9.py

In [ ]:
# bar plot back pockets


In [ ]:
# %load https://raw.githubusercontent.com/IBMDeveloperUK/pandas-workshop/master/answers/answer10.py

In [ ]:
# bar plot back pockets (seaborn)


In [ ]:
# %load https://raw.githubusercontent.com/IBMDeveloperUK/pandas-workshop/master/answers/answer11.py

<a id="extra"></a>
# 4. Optional Excercises

If you finish early:

1. Try to create other plots. Have a look at the [Pandas plot examples](https://pandas.pydata.org/pandas-docs/stable/user_guide/visualization.html) or the [Seaborn gallery](https://seaborn.pydata.org/examples/index.html) for inspiration.  
2. Have a look at these Pandas workshops and book: <br>
2.1. [Pandas workshop by Alexander Hensdorf](https://github.com/alanderex/pydata-pandas-workshop) <br>
2.2. [Pandas tutorial by Joris van den Bossche](https://github.com/jorisvandenbossche/pandas-tutorial) <br>
2.3. [Python Data Science Handbook](https://jakevdp.github.io/PythonDataScienceHandbook/) <br>
3. Try the below excercise
4. Or load one of your own datasets into a notebook and play around with the data to practice what you have learned 

## Extra excercise

A column in the `jeans` data that is containing lots of different variables is `fabric`. Cleaning this up is beyond this introduction workshop. But give it a go. Try to convert this column to columns for each fabric that will contain the percentages as numbers. For this one Google is your friend! (Disclaimer... I also have not figured this one out yet, and will need some googling to find a solution).

In [ ]:
fabric = jeans['fabric']
fabric.head()

First split this into columns:

In [ ]:
fabric = fabric.str.split(expand=True,pat = ",")
fabric.head()

Spelling mistakes can be fixed with `.replace()` 

In [ ]:
fabric = fabric.replace(to_replace=r'polyester', value=' poly', regex=True)
fabric = fabric.replace(to_replace=r'poly', value=' polyester', regex=True)
fabric = fabric.replace(to_replace=r'cottton', value='cotton', regex=True)
fabric.head()

Next steps would be to create new columns for each fabric and then merge these back into the `jeans` DataFrame. Maybe start with something like this:

In [ ]:
fabric['cotton'] = fabric[fabric.apply(lambda x: x.str.contains("cotton").any(), axis=1)]
fabric.head()

In [ ]:
fabric['cotton'] = pd.to_numeric(fabric['cotton'].replace(to_replace=r'% cotton', value='', regex=True))
fabric.head()

### Author
Margriet Groenendijk is a Data & AI Developer Advocate for IBM. She develops and presents talks and workshops about data science and AI. She is active in the local developer communities through attending, presenting and organising meetups. She has a background in climate science where she explored large observational datasets of carbon uptake by forests during her PhD, and global scale weather and climate models as a postdoctoral fellow. 

Copyright © 2019 IBM. This notebook and its source code are released under the terms of the MIT License.